In [ ]:
from imports import *
import traces_rdf
%matplotlib inline

In [ ]:
# Data files to look at -- 100 eV phonons in NF-C
hvevProd = "data/KaplanQP-updates/Kaplan-V08-09-00-100eV_51240503_000000.root"
hvev344  = "data/KaplanQP-updates/Kaplan-G4CMP-344b-100eV_51240504_000000.root"
hvev378  = "data/KaplanQP-updates/Kaplan-G4CMP-378-100eV_51240503_000000.root"
hvev50mK = "data/KaplanQP-updates/Kaplan-G4CMP-378-50mK-100eV_51240503_000000.root"
hvev379  = "data/KaplanQP-updates/Kaplan-G4CMP-379-100eV_51240503_000000.root"

In [ ]:
def loaddata(file):
    branches = ['KE','Yield','Weight','Time3']
    # NOTE: Apply 10 ms time cut to keep only phonons within readout window
    qethits = ROOT.RDataFrame(f"G4SimDir/mczip0", file, branches)\
.Filter("PType==0 & Charge==0 & Time3<10e6")\
.AsNumpy(branches)
    
    qethits['Efficiency'] = qethits['Yield']/qethits['KE']
    return qethits

In [ ]:
# Individual plots within page -- pass in data and which space to fill

def energy(qethits, subplot):
    subplot.hist(qethits['KE']*1e3,weights=qethits['Weight'],bins=500,log=True)
    subplot.set_xlabel("Phonon Energy [meV]")
    
def qetyield(qethits, subplot):
    subplot.hist(qethits['Yield']*1e3,weights=qethits['Weight'],bins=500,log=True)
    subplot.set_xlabel("QET Hit Energy [meV]")
    
def efficiency(qethits, subplot):
    subplot.hist(qethits['Efficiency'],weights=qethits['Weight'],bins=500,log=True)
    subplot.set_xlabel("QET Efficiency [meV]")
    
def power(qethits, subplot):
    subplot.hist(qethits['Time3']/1e9,weights=qethits['Weight']*qethits['Yield']*1e3,
                 bins=500,log=True)
    subplot.set_ylim([1e-2,1e5])
    subplot.set_xlabel("QET Hit Time [s]")
    subplot.set_ylabel("Energy/Bin [meV]")

In [ ]:
# Generate full page of plots from specified data file

def qetplots(file, name):
    fig, axes = subplots(2,2,figsize=(18,9), dpi = 1000)
    fig.set_tight_layout(True)
    fig.suptitle(f"100 eV Phonon-only HV-C, QET Hits (G4CMP {name})",fontsize=14)

    qethits = loaddata(file)
    energy(qethits, axes[0][0])
    qetyield(qethits, axes[0][1])
    efficiency(qethits, axes[1][0])
    power(qethits, axes[1][1])
    
    fig.savefig(f"KaplanQP-G4CMP-{name}.eps")
    fig.savefig(f"KaplanQP-G4CMP-{name}.png")

In [ ]:
qetplots(hvevProd, "V08-09-00")

In [ ]:
qetplots(hvev344, "344")

In [ ]:
qetplots(hvev378, "378")

In [ ]:
qetplots(hvev379, "379")

In [ ]:
qetplots(hvev50mK, "50mK")